Library Loading

In [14]:
import pandas as pd
import numpy as np
import io
import random as rnd
import matplotlib.pyplot as plt
import multiprocessing

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

import warnings
warnings.filterwarnings('ignore')

In [15]:
df = pd.read_csv('train.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
y = df['label']
X = df.drop(labels=['label'], axis=1)

X_train = X / 255.0
X_train = X.values.reshape(-1, 28, 28, 1)
y_train = to_categorical(y, num_classes=10)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (37800, 28, 28, 1)
X_test shape: (4200, 28, 28, 1)
y_train shape: (37800, 10)
y_test shape: (4200, 10)


In [18]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95**x, verbose=0)

### Evolutionary Algorithm (Genetic Algorithm)

In [19]:
# !pip install deap
# !pip install scoop

In [20]:
from deap import base
from deap import creator
from deap import tools

from scoop import futures

In [21]:
def getLayer():
  numberOfLayer = rnd.randint(0, 2)
  return numberOfLayer

def getNeurons():
  neurons = [8, 16, 24, 32, 48, 64]
  index = rnd.randint(0, len(neurons) - 1)   
  neuron = neurons[index]

  return neuron

def getDenseNeurons():
  neurons = [0, 32, 64, 128, 256, 512, 1024, 2048]
  index = rnd.randint(0, len(neurons) - 1)   
  neuron = neurons[index]

  return neuron

def getDropoutRate():
  rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
  index = rnd.randint(0, len(rate) - 1)   
  dropout = rate[index]

  return dropout

In [22]:
def getCNNModel(individual):
  numberOfLayer = 4
  neurons = individual[1]
  denseNeurons = individual[2]
  dropout_rate = individual[3]

  model = Sequential()

  for i in range(numberOfLayer):
    if i == 0:
      model.add(Conv2D(neurons, kernel_size=5, activation='relu', input_shape=(28, 28, 1)))
      model.add(MaxPool2D())
      model.add(Dropout(dropout_rate))

    else:
      model.add(Conv2D(neurons * i, kernel_size=5, activation='relu'))
      model.add(MaxPool2D())
      model.add(Dropout(dropout_rate))

  model.add(Flatten())
  model.add(Dense(denseNeurons, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

def evaluateModel(individual):
  model = getCNNModel(individual)
  history = model.fit(X_train, y_train, batch_size=80, epochs=20, validation_data=(X_test, y_test), callbacks=[annealer], verbose=1)
  mean = [np.mean(history.history['val_accuracy'])]

  return mean

def myMutateCNN(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_Layer()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_Neurons()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Dense_Neurons()
  
  if rnd.random() < indpb:
      individual[3] = toolbox.attribute_Dropout()

In [23]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register('attribute_Layer', getLayer)
toolbox.register('attribute_Neurons', getNeurons)
toolbox.register('attribute_Dense_Neurons', getDenseNeurons)
toolbox.register('attribute_Dropout', getDropoutRate)

function_sequence = [toolbox.attribute_Layer, toolbox.attribute_Neurons, toolbox.attribute_Dense_Neurons, toolbox.attribute_Dropout]

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [24]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 10, 15
pop = toolbox.population(n=POPSIZE) 

In [25]:
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('evaluate', evaluateModel)
toolbox.register('mutate', myMutateCNN, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)

rnd.seed(66)

In [26]:
 if __name__ == '__main__': 
  toolbox.register('map', futures.map)
  print('Start evolution of Convlution Neural Network')

  fitnesses = list(toolbox.map(toolbox.evaluate, pop))
  for individual, fitness in zip(pop, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(pop)} individuals')

  for generation in range(NGEN):
    print(f'-- Generation {generation} --')

    offspring = toolbox.select(pop, len(pop))
    offspring = list(toolbox.map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if rnd.random() < CXPB:
        c1 = toolbox.clone(child1)
        c2 = toolbox.clone(child2)
        toolbox.mate(child1, child2)
        if c1 != child1: 
          del child1.fitness.values
        if c2 != child2: 
          del child2.fitness.values

    for mutant in offspring:
      if rnd.random() < MUTPB:
        m1 = toolbox.clone(mutant)
        toolbox.mutate(mutant)
        if m1 != mutant: 
          del mutant.fitness.values

    invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
    fitnesses = list(toolbox.map(toolbox.evaluate, invalid_individual))

    for individual, fitness in zip(invalid_individual, fitnesses):
      individual.fitness.values = fitness

    print (f'  Evaluated {len(invalid_individual)} individuals')

    pop[:] = offspring
    fits = [individual.fitness.values[0] for individual in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    best_individual = tools.selBest(pop, POPSIZE)[0]
    print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
    print(f'  Min: {min(fits)}')
    print(f'  Max: {max(fits)}')

  print('End evolution of Convlution Neural Network')

  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

Start evolution of Convlution Neural Network


ValueError: Negative dimension size caused by subtracting 5 from 4 for '{{node conv2d_5/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_5/Conv2D/ReadVariableOp)' with input shapes: [?,4,4,24], [5,5,24,48].